In [21]:
import objdump as objdump
!pip install objdump
!pip install angr
!pip install pathos
!pip install tqdm



Python(88199) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Python(88204) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Python(88205) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Python(88208) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [7]:
!objdump -d data/firefox.exe > firefox_disassembly.txt

In [8]:
# Читання результату дизасемблювання
with open("firefox_disassembly.txt", "r") as f:
    disassembly = f.readlines()

# Виведення частини дизасембльованого коду
for line in disassembly[:20]:  # Перегляд перших 20 рядків
    print(line.strip())



data/firefox.exe:	file format COFF-x86-64


Disassembly of section .text:

0000000140001000 .text:
140001000: 83 fa 03                    	cmpl	$3, %edx
140001003: 74 05                       	je	5 <.text+0xa>
140001005: 85 d2                       	testl	%edx, %edx
140001007: 74 01                       	je	1 <.text+0xa>
140001009: c3                          	retq
14000100a: e9 01 00 00 00              	jmp	1 <.text+0x10>
14000100f: cc                          	int3
140001010: 41 56                       	pushq	%r14
140001012: 56                          	pushq	%rsi
140001013: 57                          	pushq	%rdi
140001014: 55                          	pushq	%rbp
140001015: 53                          	pushq	%rbx
140001016: b8 30 28 00 00              	movl	$10288, %eax


In [11]:
import angr

# Завантажте бінарний файл
binary_path = "data/firefox.exe"  # Вкажіть шлях до вашого файлу
project = angr.Project(binary_path, auto_load_libs=False)

# Ініціалізуємо початковий стан виконання з точки входу
initial_state = project.factory.entry_state()

# Створюємо менеджер симуляції для дослідження трас
simgr = project.factory.simulation_manager(initial_state)

# Виконуємо дослідження всіх можливих шляхів
simgr.explore()

# Виводимо всі знайдені траси
for deadended in simgr.deadended:
    print("Trace:")
    print(deadended.history.bbl_addrs)  # Виведення базових блоків у трасі
    print(deadended.posix.dumps(0))     # Вхідні дані, що призвели до цієї траєкторії


WARNING  | 2024-11-17 19:21:40,206 | angr.storage.memory_mixins.default_filler_mixin | The program is accessing register with an unspecified value. This could indicate unwanted behavior.
WARNING  | 2024-11-17 19:21:40,207 | angr.storage.memory_mixins.default_filler_mixin | angr will cope with this by generating an unconstrained symbolic variable and continuing. You can resolve this by:
WARNING  | 2024-11-17 19:21:40,208 | angr.storage.memory_mixins.default_filler_mixin | 1) setting a value to the initial state
WARNING  | 2024-11-17 19:21:40,209 | angr.storage.memory_mixins.default_filler_mixin | 2) adding the state option ZERO_FILL_UNCONSTRAINED_{MEMORY,REGISTERS}, to make unknown regions hold null
WARNING  | 2024-11-17 19:21:40,209 | angr.storage.memory_mixins.default_filler_mixin | 3) adding the state option SYMBOL_FILL_UNCONSTRAINED_{MEMORY,REGISTERS}, to suppress these messages.
WARNING  | 2024-11-17 19:21:40,210 | angr.storage.memory_mixins.default_filler_mixin | Filling register 

KeyboardInterrupt: 

In [ ]:
import angr
import networkx as nx
import matplotlib.pyplot as plt
from angr.exploration_techniques import LoopSeer
from pathos.multiprocessing import Pool
import logging
from tqdm import tqdm  # For progress bar

# Set up logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

# Load binary file
binary_path = "data/firefox.exe"
project = angr.Project(binary_path, auto_load_libs=False)

# Build CFG
logging.info("Building CFG...")
cfg = project.analyses.CFGFast()

# Print CFG info
logging.info(f"Number of nodes in CFG: {len(cfg.graph.nodes)}")
logging.info(f"Number of edges in CFG: {len(cfg.graph.edges)}")

# Visualize CFG (Optional)
def plot_cfg(cfg):
    logging.info("Visualizing CFG...")
    graph = nx.DiGraph()

    for node in cfg.graph.nodes:
        graph.add_node(node.addr)

    for edge in cfg.graph.edges:
        graph.add_edge(edge[0].addr, edge[1].addr)

    plt.figure(figsize=(15, 10))
    nx.draw(graph, with_labels=True, node_size=500, font_size=8)
    plt.title("Control Flow Graph")
    plt.show()

# Set up state constraints
def too_deep(state):
    return len(state.history.bbl_addrs) > 100

# Function to analyze a single address
def analyze_function(addr):
    logging.info(f"Analyzing function at address {hex(addr)}...")
    state = project.factory.entry_state(addr=addr)
    simgr = project.factory.simulation_manager(state)
    simgr.use_technique(LoopSeer(bound=10))

    # Step-by-step simulation with logging
    MAX_STEPS = 50
    step = 0
    with tqdm(total=MAX_STEPS, desc=f"Function {hex(addr)}", leave=False) as pbar:
        while len(simgr.active) > 0 and step < MAX_STEPS:
            simgr.step()
            step += 1
            pbar.update(1)
            # Log active state progress
            for state in simgr.active:
                logging.info(f"  State at address: {hex(state.addr)} | Path length: {len(state.history.bbl_addrs)}")

    logging.info(f"Completed analysis for function at address {hex(addr)}.")
    return simgr.deadended

# Gather function addresses
function_addrs = [func.addr for func in cfg.kb.functions.values()]
logging.info(f"Number of functions to analyze: {len(function_addrs)}")

# Parallel analysis with progress tracking
logging.info("Starting parallel analysis...")
results = []
with Pool(processes=4) as pool:
    with tqdm(total=len(function_addrs), desc="Analyzing Functions") as pbar:
        for result in pool.imap(analyze_function, function_addrs):
            results.append(result)
            pbar.update(1)

# Combine results
all_deadended = []
for result in results:
    all_deadended.extend(result)

# Log results
logging.info("Analysis complete. Deadended states:")
for deadended in all_deadended:
    logging.info(f"Deadended at address: {hex(deadended.addr)} | Path length: {len(deadended.history.bbl_addrs)}")


ERROR    | 2024-11-17 22:40:30,482 | angr.analyses.propagator.engine_vex.SimEnginePropagatorVEX | Unsupported Unop Iop_GetMSBs8x16.
ERROR    | 2024-11-17 22:40:32,674 | angr.analyses.propagator.engine_vex.SimEnginePropagatorVEX | Unsupported statement type CAS.
ERROR    | 2024-11-17 22:40:33,346 | angr.analyses.propagator.engine_vex.SimEnginePropagatorVEX | Unsupported statement type CAS.
ERROR    | 2024-11-17 22:40:34,569 | angr.analyses.propagator.engine_vex.SimEnginePropagatorVEX | Unsupported statement type CAS.
ERROR    | 2024-11-17 22:40:34,574 | angr.analyses.propagator.engine_vex.SimEnginePropagatorVEX | Unsupported statement type CAS.
ERROR    | 2024-11-17 22:40:34,587 | angr.analyses.propagator.engine_vex.SimEnginePropagatorVEX | Unsupported statement type CAS.
ERROR    | 2024-11-17 22:40:34,595 | angr.analyses.propagator.engine_vex.SimEnginePropagatorVEX | Unsupported statement type CAS.
ERROR    | 2024-11-17 22:40:34,606 | angr.analyses.propagator.engine_vex.SimEnginePropag

In [24]:
with open("execution_traces.txt", "w") as f:
    for deadended in simgr.deadended:
        f.write("Trace:\n")
        f.write(str(deadended.history.bbl_addrs) + "\n")  # Адреси базових блоків
        f.write(deadended.posix.dumps(0).decode('utf-8', errors='ignore') + "\n")  # Вхідні дані
